# Rebalance instances

### 1) Overall configurations 

In [1]:
import os
import sys
from pprint import pprint

import pandas as pd

sys.path.append( '../slevelsutil/' )
import round_track_util as rutil
import file_util, df_util

# Root
root_path = os.getcwd().replace("\\","/")

# Instance settings
# instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/simulation.rebalancing/instance_settings_test_rebalancing.json"
# Strong CUTS
# instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/day/enforce_sl_all_hierachical_scenarios_150.json"
# PLUS
instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/day/enforce_sl_all_hierachical_scenarios_150_plus.json"

# instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/simulation.rebalancing/no_rebalancing_fast.json"

from Instance import Instance


# Case study
test_case = "standard_vs_enforce_BB"
data_dict_path = "../data/dictionary/request_track_data_dictionary.json"

# Loading instance data
instance_case = Instance(test_case, instance_settings_path, data_dict_path)

dict_sl_status = {"MET":"Met", "UNMET":"Unmet"}

dict_sq_class = {"A":"Business", "B":"Standard", "C":"Low-cost"}
category_segmentation = pd.api.types.CategoricalDtype(categories=["Business", "Standard", "Low-cost"], ordered=True)



### 2) Loading the instance data

In [2]:
import json

def load_json(path):
    """Read json file and return dictionary"""

    # Add .json to the end of file if needed
    if path.find(".json") < 0:
        path = path + ".json"

    # Read JSON file
    with open(path) as data_file:
        data_loaded = json.load(data_file)

    return data_loaded

instances_dic = load_json(instance_settings_path)

# Folder where results will be saved
result_folder = instances_dic["result_folder"]
request_log_folder = result_folder + "/request_track"

# Folder where instances are located
instances_folder = instances_dic["instances_folder"]

# File name aggregated data
instance_name = instances_dic["instance_name"]


print("########### INSTANCE SETTINGS ##################################################")
pprint(instances_dic)

########### INSTANCE SETTINGS ##################################################
{'adjacency_matrix_file': 'C:/Users/LocalAdmin/OneDrive/leap_forward/street_network_server/tenv/data/out/manhattan/network_info/adjacency_matrix.csv',
 'distances_file': 'C:/Users/LocalAdmin/OneDrive/leap_forward/street_network_server/tenv/data/out/manhattan/distance/dist_matrix_m.csv',
 'durations_file': 'C:/Users/LocalAdmin/OneDrive/leap_forward/street_network_server/tenv/data/out/manhattan/distance/dist_matrix_duration_s.csv',
 'hiring_config': [{'name': 'method_hire_from_regional_center'}],
 'instance_description': 'Tests cases currently used in paper (with more '
                         'flexible time cuts and larger graphs), 1h from 18h, '
                         'region centers from max waiting (150)',
 'instance_name': 'PLUS',
 'instances_folder': 'C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/hour_enforce_sl_plus/',
 'labels': {'BA': 'batch_duration',
            'CD': 'contract_durat

### Get settings from instance name

E.g.:

* Input = `IN-instanceName_BA-30_TH-86400_MR-1000_IF-1000_MC-06_CD-3600-SR-S1_CS-AA_SD_VH_MO_RT_CT_UR`

* Output = 
{allow_many_to_one: True,
allow_service_deterioration: True,
allow_urgent_relocation: True,
allow_vehicle_hiring: True,
batch_duration: 30,
clear_target_list_every_round: True,
contract_duration: 3600,
customer_segmentation: AA,
initial_fleet: 1000,
max_capacity: 06,
max_requests: 1000,
reinsert_targets: True,
time_horizon: 86400}

In [3]:
def get_instance_settings(file_name):
    """ Read file name and return instance settings.
    E.g.:
     Input = IN-instanceName_BA-30_TH-86400_MR-1000_IF-1000_MC-06_CD-3600-SR-S1_CS-AA_SD_VH_MO_RT_CT_UR
     Output = {'allow_many_to_one': True,
                'allow_service_deterioration': True,
                'allow_urgent_relocation': True,
                'allow_vehicle_hiring': True,
                'batch_duration': '30',
                'clear_target_list_every_round': True,
                'contract_duration': '3600',
                'customer_segmentation': 'AA',
                'initial_fleet': '1000',
                'max_capacity': '06',
                'max_requests': '1000',
                'reinsert_targets': True,
                'time_horizon': '86400'}
    """
    label_setting_dic = instances_dic["labels"]
    
    print(file_name)
        
    # E.g., ['BA-30', 'TH-86400', 'MR-1000', 'IF-1000', 'MC-06', 'CD-3600-SR-S1', 'CS-AA', 'SD', 'VH', 'MO', 'RT', 'CT', 'UR']
    file_instances = file_name.split("_")

    instance_settings = dict()

    for e in file_instances:
        
        if e in label_setting_dic.keys():
            # E.g., e =  SD
            k = label_setting_dic[e]
            # E.g., k = allow_service_deterioration
            instance_settings[k] = True
            
        else:
            # E.g., lv = ["BA", "30"]
            lv  = e.split('-')
            # E.g., e2 = BA
            e2 = lv[0]
            # E.g., k = batch_duration
            k2 = label_setting_dic[e2]
            
            if len(lv) > 1:
                 # E.g., v = '30'
                v = lv[1]
                instance_settings[k2] = v
            else:
                # label is not in instance name = False
                instance_settings[k] = False

    return instance_settings

### Aggregate results (folder round_track)

Instance fields in `round_track` folder. Every line is a snapshot of a simulation round of 30 seconds (first column is round `timestamp`):

#### Request status per round
* `waiting`
* `finished`
* `denied`
* `n_requests`

#### Freelance vehicles per round
* `hired_vehicles`
* `deactivated_vehicles`

#### Vehicle status per round
* `active_vehicles`
* `enroute_count`
* `parked_vehicles`
* `origin_vehicles`
* `simulation.rebalancing`
* `stopped_rebalancing`
* `idle`
* `picking_up`
* `O1,O2,O3,O4`
* `V1,V2,V3,V4`
* `distance_traveled_cruising`
* `distance_traveled_loaded`
* `distance_traveled_rebalancing`
* `run_time`

#### Vehicle status per round (seats)
* `seat_count`
* `picking_up_seats`
* `rebalancing_seats`
* `empty_seats`
* `total_capacity`

#### Service quality
* `pk_delay`
* `total_delay`
* `A_pk,A_dp,A_count,A_unmet_slevels`
* `B_pk,B_dp,B_count,B_unmet_slevels`
* `C_pk,C_dp,C_count,C_unmet_slevels`

### Aggregate results (folder request_track)

* `earliest`
* `id` = 1, 2, 3, ..., #USERS
* `class` = A, B, C
* `pk_delay`
* `ride_delay`
* `pk_time`
* `dp_time`
* `id_from` = Network id
* `id_to` = Network id
* `dist` = trip(id_from, id_to) in seconds
* `service` = {FLEET, FREELANCE}
* `service_level` = {MET, UNMET}

import pandas as pd
from pprint import pprint

def get_request_track_dic(path_experiment, name_experiment):
    
    # Load results
    experiment_file = "{}request_track/{}.csv".format(path_experiment, name_experiment)
    
    # print("Processing experiment file '{}'".format(experiment_file))
    df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)
    
    aggfunc = {"pk_delay" : ['mean', 'count', 'max']}
    
    dfp = df.pivot_table(index="class", columns="service_level", aggfunc=aggfunc, values=["pk_delay"])

    return dfp

In [4]:

def get_request_track_dic(path_experiment, name_experiment):
    
    # Load results
    experiment_file = "{}request_track/{}.csv".format(path_experiment, name_experiment)
    
    # print("Processing experiment file '{}'".format(experiment_file))
    df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)
    
    service_quality_dic = dict()
 
    for service_level in ["MET", "UNMET"]:
        total = df[(df['service_level'] == service_level)]["pk_delay"].count()
        for sq_class in ['A', 'B', 'C']:
            filter_sq_sl = (df['class'] == sq_class) & (df['service_level'] == service_level)
            #service_quality_dic["{}_{}".format(service_level, sq_class)] = df.loc[filter_sq_sl]["pk_delay"].count()
            if service_level == 'MET':
                service_quality_dic["{}_{}".format(service_level, sq_class)] = df.loc[filter_sq_sl]["pk_delay"].mean()
            else:
                service_quality_dic["{}_{}".format(service_level, sq_class)] = df.loc[filter_sq_sl]["pk_delay"].count()
                
        service_quality_dic["{}_TOTAL".format(service_level)] = total
    
    return service_quality_dic

### Processing single instance

In [5]:
import pandas as pd

def get_aggregate_request_dic(path_experiment, name_experiment):
    
    # Load results
    experiment_file = "{}request_track/{}.csv".format(path_experiment, name_experiment)
    
   
    df = pd.read_csv(experiment_file, index_col = "earliest",  parse_dates = True)

    return df

# Rebalance and no rebalance aggregate data

In [6]:
# Get all instances in folder
instance_file_names = os.listdir(request_log_folder)

instance_file_names = [i for i in instance_file_names if instances_dic['instance_name'] in i]

# Instance Info (key = instance name)
dic_all = dict()
print("Reading files in folder:", request_log_folder)

count = 1
for file_name in instance_file_names:
    
    
        
    instance, extension = file_name.split(".")
    
    # Instance settings
    instance_settings_dic = get_instance_settings(instance)
    # if instance_settings_dic['instance_name'] != 'WEEKDENY' or instance_settings_dic['customer_segmentation'] in ['A', 'B', 'C']:
    #     continue
        
    print("  - Processing", instance)
        
    count = count + 1
    round_track_agg_dic = rutil.get_results_dic(instances_folder, instance)
    request_track_agg_dic = {}#get_request_track_dic(instances_folder, instance)
    
    # Get aggregated results
    dic_all[instance] = {**instance_settings_dic, **round_track_agg_dic, **request_track_agg_dic}
    

Reading files in folder: C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/hour_enforce_sl_plus//request_track
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S0_VH_SD_RE-OP_OPT-ERTV
  - Processing IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S0_VH_SD_RE-OP_OPT-ERTV
Index(['waiting', 'finished', 'denied', 'n_requests', 'seat_count',
       'picking_up_seats', 'rebalancing_seats', 'empty_seats',
       'total_capacity', 'active_vehicles', 'hired_vehicles',
       'deactivated_vehicles', 'enroute_count', 'pk_delay', 'total_delay',
       'parked_vehicles', 'origin_vehicles', 'rebalancing',
       'stopped_rebalancing', 'idle', 'picking_up', 'O1', 'O2', 'O3', 'O4',
       'V1', 'V2', 'V3', 'V4', 'distance_traveled_cruising',
       'distance_traveled_loaded', 'distance_traveled_rebalancing',
       'time_ride_matching_s', 'time_update_fleet_status_s',
       'time_vehicle_rebalancing_s', 'A_pk', 'A_dp', 'A_count',
       'A_unmet_slevels', 'B_pk',

../slevelsutil\round_track_util.py:176: RuntimeWarning: invalid value encountered in double_scalars
  "distance_cruising": "{:.2%}".format(distance_cruising / distance_total),
../slevelsutil\round_track_util.py:177: RuntimeWarning: invalid value encountered in double_scalars
  "distance_loaded": "{:.2%}".format(distance_loaded / distance_total),
../slevelsutil\round_track_util.py:178: RuntimeWarning: invalid value encountered in double_scalars
  "distance_rebalancing": "{:.2%}".format(distance_rebalancing / distance_total),


In [7]:
df_all = pd.DataFrame.from_dict(dic_all, orient='index')
print(df_all.columns)
df_all.to_csv("rebal_and_no_rebal.csv")

df_all = df_all.rename(columns={"rebalance_optimal_alonso_mora":"matching"})

df_all

Index(['instance_name', 'maximal_hiring_delay', 'batch_duration',
       'simulation_time', 'max_requests', 'initial_fleet', 'max_capacity',
       'customer_segmentation', 'contract_duration', 'service_rate',
       'allow_vehicle_hiring', 'allow_service_deterioration', 'rebalance',
       'matching', 'serviced_seats', 'picking_up_seats', 'rebalancing_seats',
       'parked_seats', 'serviced', 'denied', 'max_hired', 'occupancy', 'o1',
       'o2', 'o3', 'o4', 'v1', 'v2', 'v3', 'v4', 'mean_hired', 'median_hired',
       'avg_seats', 'max_seats', 'id_max_seats', 'median_seats', 'mean_active',
       'total_requests', 'avg_pk_delay', 'avg_ride_delay', 'total_runtime_s',
       'avg_runtime_s', 'distance_cruising', 'distance_loaded',
       'distance_rebalancing', 'distance_total'],
      dtype='object')


,instance_name,maximal_hiring_delay,batch_duration,simulation_time,max_requests,initial_fleet,max_capacity,customer_segmentation,contract_duration,service_rate,...,mean_active,total_requests,avg_pk_delay,avg_ride_delay,total_runtime_s,avg_runtime_s,distance_cruising,distance_loaded,distance_rebalancing,distance_total
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S0_VH_SD_RE-OP_OPT-ERTV,PLUS,150,30,3600,1000,1000,4,AA,0,S0,...,759.750000,14234.0,121.17,149.31,2972.13,15.81,nan%,nan%,nan%,0.0
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S0_VH_SD_RE-OP_OPT-JAVIER,PLUS,150,30,3600,1000,1000,4,AA,0,S0,...,550.591837,14234.0,228.84,263.80,21289.24,108.62,nan%,nan%,nan%,0.0
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S0_VH_SD_RE-OP_OPT-JAVIERSL,PLUS,150,30,3600,1000,1000,4,AA,0,S0,...,597.376344,14234.0,193.15,228.27,12871.69,69.20,nan%,nan%,nan%,0.0
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S10_VH_SD_RE-OP_OPT-ERTV,PLUS,150,30,3600,1000,1000,4,AA,0,S10,...,759.840426,14234.0,122.36,149.87,3153.85,16.78,nan%,nan%,nan%,0.0
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S10_VH_SD_RE-OP_OPT-JAVIERSL,PLUS,150,30,3600,1000,1000,4,AA,0,S10,...,581.020942,14234.0,193.14,228.48,10729.68,56.18,nan%,nan%,nan%,0.0
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S8_VH_SD_RE-OP_OPT-ERTV,PLUS,150,30,3600,1000,1000,4,AA,0,S8,...,722.239796,14234.0,136.80,166.52,6993.08,35.68,nan%,nan%,nan%,0.0
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S8_VH_SD_RE-OP_OPT-JAVIERSL,PLUS,150,30,3600,1000,1000,4,AA,0,S8,...,595.768817,14234.0,193.80,228.82,11992.77,64.48,nan%,nan%,nan%,0.0
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S9_VH_SD_RE-OP_OPT-ERTV,PLUS,150,30,3600,1000,1000,4,AA,0,S9,...,764.795699,14234.0,128.34,157.75,5851.98,31.46,nan%,nan%,nan%,0.0
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S9_VH_SD_RE-OP_OPT-JAVIERSL,PLUS,150,30,3600,1000,1000,4,AA,0,S9,...,578.104167,14234.0,194.43,229.75,11291.58,58.81,nan%,nan%,nan%,0.0
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-BB_CD-0_SR-S0_VH_SD_RE-OP_OPT-ERTV,PLUS,150,30,3600,1000,1000,4,BB,0,S0,...,570.207071,14234.0,132.81,204.33,6580.62,33.24,nan%,nan%,nan%,0.0


In [8]:
instance_case.headers


indexes = [
    "customer_segmentation",
    "service_rate",
    # "maximal_hiring_delay",
    #"matching"
]

### Run times

In [9]:
df_time = df_all.copy()

# filter_service_rate = (df_time['service_rate']!="S0")
filter_maximal_hiring_delay = (df_time['maximal_hiring_delay'].isin(["150", "1", "2"]))

all_filters = (
        # filter_service_rate &
        filter_maximal_hiring_delay
)
df_time = df_time[all_filters]

indexes_time = [
    "customer_segmentation",
    "service_rate",
    "matching"
]

values = [
    'total_runtime_s',
    'avg_runtime_s'
]


df_time = df_time[indexes_time + values]
df_time

,customer_segmentation,service_rate,matching,total_runtime_s,avg_runtime_s
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S0_VH_SD_RE-OP_OPT-ERTV,AA,S0,ERTV,2972.13,15.81
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S0_VH_SD_RE-OP_OPT-JAVIER,AA,S0,JAVIER,21289.24,108.62
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S0_VH_SD_RE-OP_OPT-JAVIERSL,AA,S0,JAVIERSL,12871.69,69.20
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S10_VH_SD_RE-OP_OPT-ERTV,AA,S10,ERTV,3153.85,16.78
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S10_VH_SD_RE-OP_OPT-JAVIERSL,AA,S10,JAVIERSL,10729.68,56.18
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S8_VH_SD_RE-OP_OPT-ERTV,AA,S8,ERTV,6993.08,35.68
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S8_VH_SD_RE-OP_OPT-JAVIERSL,AA,S8,JAVIERSL,11992.77,64.48
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S9_VH_SD_RE-OP_OPT-ERTV,AA,S9,ERTV,5851.98,31.46
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S9_VH_SD_RE-OP_OPT-JAVIERSL,AA,S9,JAVIERSL,11291.58,58.81
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-BB_CD-0_SR-S0_VH_SD_RE-OP_OPT-ERTV,BB,S0,ERTV,6580.62,33.24


In [10]:


df_time = instance_case.rename_values(df_time)
df_time = instance_case.rename_headers(df_time)
df_time = instance_case.apply_categories_columns(df_time, indexes_time)
df_time = df_time.sort_values(by=[instance_case.headers[i] for i in indexes_time])
df_time = instance_case.set_index(df_time, indexes_time)
df_time.to_csv("runtime.csv")
df_time

Total time (s) Avg. round time (s)
User base Service rate Policy                                              
B+        0%           Min. waiting            21289.24              108.62
                       Enforce SL              12871.69               69.20
                       Enforce SL + Hire        2972.13               15.81
          80%          Enforce SL              11992.77               64.48
                       Enforce SL + Hire        6993.08               35.68
          90%          Enforce SL              11291.58               58.81
                       Enforce SL + Hire        5851.98               31.46
          100%         Enforce SL              10729.68               56.18
                       Enforce SL + Hire        3153.85               16.78
S+        0%           Min. waiting            24505.63              126.32
                       Enforce SL              18386.51               93.81
                       Enforce SL + Hire        6580.62               33.24
          80%          Enforce SL              19293.13               98.43
                       Enforce SL + Hire       23943.19              119.72
          90%          Enforce SL              18452.69               95.61
                       Enforce SL + Hire       22525.65              119.82
          100%         Enforce SL              16928.43               87.71
                       Enforce SL + Hire        6047.36               30.85
L+        0%           Min. waiting            25109.09              132.85
                       Enforce SL              20094.24              103.58
                       Enforce SL + Hire       15039.98               76.73
          80%          Enforce SL              19271.08               98.83
                       Enforce SL + Hire       23657.90              116.54
          90%          Enforce SL              18681.01               97.81
                       Enforce SL + Hire       23333.19              119.05
          100%         Enforce SL              16926.03               88.16
                       Enforce SL + Hire       14348.55               74.73

In [11]:
df_latex = df_time.reset_index()
df_latex["space1"] = ""
df_latex

,User base,Service rate,Policy,Total time (s),Avg. round time (s),space1
0,B+,0%,Min. waiting,21289.24,108.62,
1,B+,0%,Enforce SL,12871.69,69.20,
2,B+,0%,Enforce SL + Hire,2972.13,15.81,
3,B+,80%,Enforce SL,11992.77,64.48,
4,B+,80%,Enforce SL + Hire,6993.08,35.68,
5,B+,90%,Enforce SL,11291.58,58.81,
6,B+,90%,Enforce SL + Hire,5851.98,31.46,
7,B+,100%,Enforce SL,10729.68,56.18,
8,B+,100%,Enforce SL + Hire,3153.85,16.78,
9,S+,0%,Min. waiting,24505.63,126.32,


In [12]:
order_cols = (
    instance_case.get_headers_from_tags(indexes_time)
    + ["space1"]
    + instance_case.get_headers_from_tags(values)
)
order_cols

['User base',
 'Service rate',
 'Policy',
 'space1',
 'Total time (s)',
 'Avg. round time (s)']

In [13]:
df_latex = df_latex[order_cols]
df_latex

,User base,Service rate,Policy,space1,Total time (s),Avg. round time (s)
0,B+,0%,Min. waiting,,21289.24,108.62
1,B+,0%,Enforce SL,,12871.69,69.20
2,B+,0%,Enforce SL + Hire,,2972.13,15.81
3,B+,80%,Enforce SL,,11992.77,64.48
4,B+,80%,Enforce SL + Hire,,6993.08,35.68
5,B+,90%,Enforce SL,,11291.58,58.81
6,B+,90%,Enforce SL + Hire,,5851.98,31.46
7,B+,100%,Enforce SL,,10729.68,56.18
8,B+,100%,Enforce SL + Hire,,3153.85,16.78
9,S+,0%,Min. waiting,,24505.63,126.32


In [14]:
centering = "".join(len(df_latex.columns)*["c"])
df_latex = df_latex.sort_values(by=[instance_case.headers[i] for i in indexes_time])
df_latex = instance_case.set_index(df_latex, indexes_time)
df_latex

space1 Total time (s)  \
User base Service rate Policy                                    
B+        0%           Min. waiting                   21289.24   
                       Enforce SL                     12871.69   
                       Enforce SL + Hire               2972.13   
          80%          Enforce SL                     11992.77   
                       Enforce SL + Hire               6993.08   
          90%          Enforce SL                     11291.58   
                       Enforce SL + Hire               5851.98   
          100%         Enforce SL                     10729.68   
                       Enforce SL + Hire               3153.85   
S+        0%           Min. waiting                   24505.63   
                       Enforce SL                     18386.51   
                       Enforce SL + Hire               6580.62   
          80%          Enforce SL                     19293.13   
                       Enforce SL + Hire              23943.19   
          90%          Enforce SL                     18452.69   
                       Enforce SL + Hire              22525.65   
          100%         Enforce SL                     16928.43   
                       Enforce SL + Hire               6047.36   
L+        0%           Min. waiting                   25109.09   
                       Enforce SL                     20094.24   
                       Enforce SL + Hire              15039.98   
          80%          Enforce SL                     19271.08   
                       Enforce SL + Hire              23657.90   
          90%          Enforce SL                     18681.01   
                       Enforce SL + Hire              23333.19   
          100%         Enforce SL                     16926.03   
                       Enforce SL + Hire              14348.55   

                                         Avg. round time (s)  
User base Service rate Policy                                 
B+        0%           Min. waiting                   108.62  
                       Enforce SL                      69.20  
                       Enforce SL + Hire               15.81  
          80%          Enforce SL                      64.48  
                       Enforce SL + Hire               35.68  
          90%          Enforce SL                      58.81  
                       Enforce SL + Hire               31.46  
          100%         Enforce SL                      56.18  
                       Enforce SL + Hire               16.78  
S+        0%           Min. waiting                   126.32  
                       Enforce SL                      93.81  
                       Enforce SL + Hire               33.24  
          80%          Enforce SL                      98.43  
                       Enforce SL + Hire              119.72  
          90%          Enforce SL                      95.61  
                       Enforce SL + Hire              119.82  
          100%         Enforce SL                      87.71  
                       Enforce SL + Hire               30.85  
L+        0%           Min. waiting                   132.85  
                       Enforce SL                     103.58  
                       Enforce SL + Hire               76.73  
          80%          Enforce SL                      98.83  
                       Enforce SL + Hire              116.54  
          90%          Enforce SL                      97.81  
                       Enforce SL + Hire              119.05  
          100%         Enforce SL                      88.16  
                       Enforce SL + Hire               74.73

In [15]:
print(file_util.df_to_latex(df_latex, column_format=centering))

\begin{tabular}{cccccc}
\toprule
   &      &                   & space1 & Total time (s) & Avg. round time (s) \\
User base & Service rate & Policy &        &                &                     \\
\midrule
\multirow{9}{*}{B+} & \multirow{3}{*}{0\%} & Min. waiting &        &       21289.24 &              108.62 \\
   &      & Enforce SL &        &       12871.69 &               69.20 \\
   &      & Enforce SL + Hire &        &        2972.13 &               15.81 \\
\cline{2-6}
   & \multirow{2}{*}{80\%} & Enforce SL &        &       11992.77 &               64.48 \\
   &      & Enforce SL + Hire &        &        6993.08 &               35.68 \\
\cline{2-6}
   & \multirow{2}{*}{90\%} & Enforce SL &        &       11291.58 &               58.81 \\
   &      & Enforce SL + Hire &        &        5851.98 &               31.46 \\
\cline{2-6}
   & \multirow{2}{*}{100\%} & Enforce SL &        &       10729.68 &               56.18 \\
   &      & Enforce SL + Hire &        &        3153.85 

## Seat occupation

In [16]:
df_seats = df_all.copy()
filter_service_rate = (df_seats['service_rate']!="S0")
filter_method = (df_seats['matching']=="ERTV")
filter_maximal_hiring_delay = (df_seats['maximal_hiring_delay']=="150")

all_filters = filter_service_rate & filter_method & filter_maximal_hiring_delay

df_seats = df_seats[all_filters]


v_cols = [
    "v1","v2","v3","v4",
]
v_summary = [
    "mean_hired","median_hired","max_hired"
]

o_cols = [
    "o1","o2","o3","o4",
    #"median_seats"
]

o_summary = [
    "avg_seats","max_seats",
]

cols = v_cols + v_summary

df_seats = df_seats[indexes + cols]

df_util.set_columns_to_int(df_seats, {"max_hired", "max_seats", "median_seats", "median_hired"}.intersection(cols))

df_seats = instance_case.rename_values(df_seats)
df_seats = instance_case.rename_headers(df_seats)
df_seats = instance_case.apply_categories_columns(df_seats, indexes)
df_seats = df_seats.sort_values(by=[instance_case.headers[i] for i in indexes])
df_seats = instance_case.set_index(df_seats, indexes)

df_seats

1       2      3      4    Avg.  Median  Max.
User base Service rate                                                    
B+        80%           76.39%  19.41%  2.74%  1.46%  223.88     309   493
          90%           79.67%  15.79%  3.21%  1.32%  243.75     336   497
          100%          79.92%  14.75%  4.16%  1.17%  254.74     355   520
S+        80%           76.41%  15.48%  4.82%  3.29%   32.79      22   102
          90%           73.45%  17.40%  5.45%  3.70%   47.41      43   168
          100%          74.29%  18.57%  4.94%  2.20%   60.32      65   167
L+        80%           66.49%  26.89%  3.36%  3.26%   15.64       8    72
          90%           77.33%  17.55%  3.50%  1.62%   30.85      22   104
          100%          76.71%  18.01%  2.61%  2.67%   56.07      51   191

Adding spaces for latex formatting

In [17]:
df_latex = df_seats.reset_index()
df_latex["space1"] = ""
df_latex["space2"] = ""

order_cols = (
    instance_case.get_headers_from_tags(indexes)
    + ["space1"]
    + instance_case.get_headers_from_tags(v_cols)
    + ["space2"]
    + instance_case.get_headers_from_tags(v_summary)
)

df_latex = df_latex[order_cols]

df_latex = df_latex.sort_values(by=[instance_case.headers[i] for i in indexes])
df_latex = instance_case.set_index(df_latex, indexes)
df_latex

space1       1       2      3      4 space2    Avg.  \
User base Service rate                                                       
B+        80%                  76.39%  19.41%  2.74%  1.46%         223.88   
          90%                  79.67%  15.79%  3.21%  1.32%         243.75   
          100%                 79.92%  14.75%  4.16%  1.17%         254.74   
S+        80%                  76.41%  15.48%  4.82%  3.29%          32.79   
          90%                  73.45%  17.40%  5.45%  3.70%          47.41   
          100%                 74.29%  18.57%  4.94%  2.20%          60.32   
L+        80%                  66.49%  26.89%  3.36%  3.26%          15.64   
          90%                  77.33%  17.55%  3.50%  1.62%          30.85   
          100%                 76.71%  18.01%  2.61%  2.67%          56.07   

                        Median  Max.  
User base Service rate                
B+        80%              309   493  
          90%              336   497  
          100%             355   520  
S+        80%               22   102  
          90%               43   168  
          100%              65   167  
L+        80%                8    72  
          90%               22   104  
          100%              51   191

In [18]:
print(file_util.df_to_latex(df_latex, column_format="ccccccccccc"))

\begin{tabular}{ccccccccccc}
\toprule
   &      & space1 &       1 &       2 &      3 &      4 & space2 &    Avg. &  Median &  Max. \\
User base & Service rate &        &         &         &        &        &        &         &         &       \\
\midrule
\multirow{3}{*}{B+} & 80\% &        &  76.39\% &  19.41\% &  2.74\% &  1.46\% &        &  223.88 &     309 &   493 \\
   & 90\% &        &  79.67\% &  15.79\% &  3.21\% &  1.32\% &        &  243.75 &     336 &   497 \\
   & 100\% &        &  79.92\% &  14.75\% &  4.16\% &  1.17\% &        &  254.74 &     355 &   520 \\
\cline{1-11}
\multirow{3}{*}{S+} & 80\% &        &  76.41\% &  15.48\% &  4.82\% &  3.29\% &        &   32.79 &      22 &   102 \\
   & 90\% &        &  73.45\% &  17.40\% &  5.45\% &  3.70\% &        &   47.41 &      43 &   168 \\
   & 100\% &        &  74.29\% &  18.57\% &  4.94\% &  2.20\% &        &   60.32 &      65 &   167 \\
\cline{1-11}
\multirow{3}{*}{L+} & 80\% &        &  66.49\% &  26.89\% &  3.36\% &  3.26\

In [19]:
b = df_all[df_all['instance_name']=='WEEKDENY']

b = df_all.copy()

key_cs = 'customer_segmentation' # (A, AA, BB, etc.)
key_reb = 'rebalance'

# b = b[[
#     'rebalance',
#     'customer_segmentation',
#     'customer_segmentation',
#     'mean_active',
#     'occupancy',
#     'distance_cruising',
#     'distance_loaded',
#     'distance_rebalancing',
#     'distance_total'
# ]+["{}_{}".format(service_level, sq_class) for sq_class in ['A', 'B', 'C'] for service_level in ["MET", "UNMET"]]+ ['MET_TOTAL', 'UNMET_TOTAL']]

# Filtering data
b = b[b[key_cs].isin(["AA", "BB", "CC"])]

# convert just columns "a" and "b"
#a[['contract_duration', 'median_seats', 'distance_total']] = a[['contract_duration', 'median_seats', 'distance_total']].apply(pd.to_numeric)

b = b.astype({'distance_total':int})
# Establishing category order and alias dictionaries
dict_segmentation = {"AA":"B+", "BB":"S+", "CC":"L+", "A":"B", "B":"S", "C":"L"}
category_segmentation = pd.api.types.CategoricalDtype(categories=["B+", "S+", "L+", "B", "S", "L"], ordered=True)

dict_service_rate = {"S1":"SR1", "S2":"SR2", "S3":"SR3"}
category_service_rate = pd.api.types.CategoricalDtype(categories=["SR1", "SR2", "SR3"], ordered=True)


# Renaming data and applying aliases
b[key_cs] = b[key_cs].map(lambda e:dict_segmentation[e])
b[key_cs] = b[key_cs].astype(category_segmentation)

b[key_reb] = b[key_reb].map(lambda e:( "YES" if e == True else "NO"))

b = b.sort_values(by=[key_reb, key_cs])

b.rename(columns={'customer_segmentation': 'Customer segmentation',
                    'rebalance': 'Rebalance',
                    'mean_active': '#Active vehicles/Round',
                    'occupancy': 'Occupancy/Round',
                    'distance_cruising': 'Cruising',
                    'distance_rebalancing': 'Rebalancing',
                    'distance_loaded': 'Servicing',
                    'distance_total': 'Total (Km)'}, inplace=True)

b = b.set_index([ 'Rebalance' , 'Customer segmentation'])

#b['#Active vehicles/Round'] = b['#Active vehicles/Round'].apply(lambda x:"{:.2f}".format(x))

#b = b[['#Active vehicles/Round', 'Occupancy/Round', 'MET_A', 'MET_B', 'MET_C', 'MET_TOTAL', 'UNMET_A', 'UNMET_B', 'UNMET_C', 'UNMET_TOTAL',  'Cruising', 'Servicing', 'Rebalancing', 'Total (Km)'] ]
#b["Service rate"] = (b['MET_TOTAL']/(b['MET_TOTAL'] + b['UNMET_TOTAL'])).apply(lambda x: "{:.2%}".format(x))
b

instance_name maximal_hiring_delay  \
Rebalance Customer segmentation                                      
NO        B+                             PLUS                  150   
          B+                             PLUS                  150   
          B+                             PLUS                  150   
          B+                             PLUS                  150   
          B+                             PLUS                  150   
          B+                             PLUS                  150   
          B+                             PLUS                  150   
          B+                             PLUS                  150   
          B+                             PLUS                  150   
          S+                             PLUS                  150   
          S+                             PLUS                  150   
          S+                             PLUS                  150   
          S+                             PLUS                  150   
          S+                             PLUS                  150   
          S+                             PLUS                  150   
          S+                             PLUS                  150   
          S+                             PLUS                  150   
          S+                             PLUS                  150   
          L+                             PLUS                  150   
          L+                             PLUS                  150   
          L+                             PLUS                  150   
          L+                             PLUS                  150   
          L+                             PLUS                  150   
          L+                             PLUS                  150   
          L+                             PLUS                  150   
          L+                             PLUS                  150   
          L+                             PLUS                  150   

                                batch_duration simulation_time max_requests  \
Rebalance Customer segmentation                                               
NO        B+                                30            3600         1000   
          B+                                30            3600         1000   
          B+                                30            3600         1000   
          B+                                30            3600         1000   
          B+                                30            3600         1000   
          B+                                30            3600         1000   
          B+                                30            3600         1000   
          B+                                30            3600         1000   
          B+                                30            3600         1000   
          S+                                30            3600         1000   
          S+                                30            3600         1000   
          S+                                30            3600         1000   
          S+                                30            3600         1000   
          S+                                30            3600         1000   
          S+                                30            3600         1000   
          S+                                30            3600         1000   
          S+                                30            3600         1000   
          S+                                30            3600         1000   
          L+                                30            3600         1000   
          L+                                30            3600         1000   
          L+                                30            3600         1000   
          L+                                30            3600         1000   
          L+                                30            3600         1000   
          L+              

### Service info (Rebalancing X No Rebalancing)

### Fleet operation info (Rebalancing X No Rebalancing)

In [20]:
df_fleet = b[['Occupancy/Round', '#Active vehicles/Round', 'Cruising', 'Servicing', 'Rebalancing', 'Total (Km)']]
df_fleet['#Active vehicles/Round'] = (df_fleet['#Active vehicles/Round'].astype(float)/1000).apply(lambda x: "{:.2%}".format(x))
print(df_fleet.to_latex(multicolumn=True, multirow=True, column_format='ccccccc'))
df_fleet

\begin{tabular}{ccccccc}
\toprule
   &    & Occupancy/Round & \#Active vehicles/Round & Cruising & Servicing & Rebalancing &  Total (Km) \\
Rebalance & Customer segmentation &                 &                        &          &           &             &             \\
\midrule
\multirow{27}{*}{NO} & B+ &          26.37\% &                 75.98\% &     nan\% &      nan\% &        nan\% &           0 \\
   & B+ &          23.63\% &                 55.06\% &     nan\% &      nan\% &        nan\% &           0 \\
   & B+ &          25.33\% &                 59.74\% &     nan\% &      nan\% &        nan\% &           0 \\
   & B+ &          26.36\% &                 75.98\% &     nan\% &      nan\% &        nan\% &           0 \\
   & B+ &          24.56\% &                 58.10\% &     nan\% &      nan\% &        nan\% &           0 \\
   & B+ &          25.65\% &                 72.22\% &     nan\% &      nan\% &        nan\% &           0 \\
   & B+ &          25.20\% &              

C:\Users\LocalAdmin\anaconda3\envs\env_slevels\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Occupancy/Round #Active vehicles/Round  \
Rebalance Customer segmentation                                          
NO        B+                             26.37%                 75.98%   
          B+                             23.63%                 55.06%   
          B+                             25.33%                 59.74%   
          B+                             26.36%                 75.98%   
          B+                             24.56%                 58.10%   
          B+                             25.65%                 72.22%   
          B+                             25.20%                 59.58%   
          B+                             26.94%                 76.48%   
          B+                             24.53%                 57.81%   
          S+                             30.80%                 57.02%   
          S+                             29.15%                 51.97%   
          S+                             30.68%                 54.09%   
          S+                             31.27%                 57.54%   
          S+                             31.18%                 55.08%   
          S+                             30.44%                 55.17%   
          S+                             30.54%                 54.27%   
          S+                             32.16%                 59.54%   
          S+                             31.09%                 55.07%   
          L+                             30.23%                 56.56%   
          L+                             29.95%                 53.65%   
          L+                             30.06%                 54.09%   
          L+                             30.79%                 57.97%   
          L+                             30.55%                 54.95%   
          L+                             28.91%                 52.84%   
          L+                             29.98%                 54.06%   
          L+                             29.81%                 55.36%   
          L+                             30.50%                 55.11%   

                                Cruising Servicing Rebalancing  Total (Km)  
Rebalance Customer segmentation                                             
NO        B+                        nan%      nan%        nan%           0  
          B+                        nan%      nan%        nan%           0  
          B+                        nan%      nan%        nan%           0  
          B+                        nan%      nan%        nan%           0  
          B+                        nan%      nan%        nan%           0  
          B+                        nan%      nan%        nan%           0  
          B+                        nan%      nan%        nan%           0  
          B+                        nan%      nan%        nan%           0  
          B+                        nan%      nan%        nan%           0  
          S+                        nan%      nan%        nan%           0  
          S+                        nan%      nan%        nan%           0  
          S+                        nan%      nan%        nan%           0  
          S+                        nan%      nan%        nan%           0  
          S+                        nan%      nan%        nan%           0  
          S+                        nan%      nan%        nan%           0  
          S+                        nan%      nan%        nan%           0  
          S+                        nan%      nan%        nan%           0  
          S+                        nan%      nan%        nan%           0  
          L+                        nan%      nan%        nan%           0  
          L+                        nan%      nan%        nan%           0  
          L+                        nan%      nan%        nan%           0  
          L+                        nan%      nan%        nan%           0  
          L+                        n

# Hiring aggregate data - Fleet stats

### Processing all instances in folder

In [21]:
# Get all instances in folder
instance_file_names = file_util.read_files_from_folder(request_log_folder)


# Instance Info (key = instance name)

print("Reading files in folder:", request_log_folder)
dic_all_round_track = dict()

for file_name in instance_file_names:
    
    instance, extension = file_name.split(".")
        
    # Instance settings
    instance_settings_dic = get_instance_settings(instance)
    
    if instance_settings_dic['instance_name'] == "HIRINGWEEKMAXWAITINGREB2" or 'rebalance' not in instance_settings_dic.keys():
        continue
    if instance_settings_dic['customer_segmentation'] in ['A', 'B', 'C']:
        continue
  
    #if instance_settings_dic['instance_name'] != 'HIRINGWEEKMAXWAITING':
    #    continue
        
    print("  - Processing", instance)
    
    round_track_agg_dic = rutil.get_results_dic(instances_folder, instance)
    
    # Get aggregated results
    dic_all_round_track[instance] = {**instance_settings_dic, **round_track_agg_dic}

#print("Instance settings:")
#pprint(dic_instance_settings)

Reading 27 files from 'C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/hour_enforce_sl_plus//request_track'.
Reading files in folder: C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/hour_enforce_sl_plus//request_track
IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S0_VH_SD_RE-OP_OPT-ERTV
  - Processing IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S0_VH_SD_RE-OP_OPT-ERTV
Index(['waiting', 'finished', 'denied', 'n_requests', 'seat_count',
       'picking_up_seats', 'rebalancing_seats', 'empty_seats',
       'total_capacity', 'active_vehicles', 'hired_vehicles',
       'deactivated_vehicles', 'enroute_count', 'pk_delay', 'total_delay',
       'parked_vehicles', 'origin_vehicles', 'rebalancing',
       'stopped_rebalancing', 'idle', 'picking_up', 'O1', 'O2', 'O3', 'O4',
       'V1', 'V2', 'V3', 'V4', 'distance_traveled_cruising',
       'distance_traveled_loaded', 'distance_traveled_rebalancing',
       'time_ride_matching_s', 'time_upd

../slevelsutil\round_track_util.py:176: RuntimeWarning: invalid value encountered in double_scalars
  "distance_cruising": "{:.2%}".format(distance_cruising / distance_total),
../slevelsutil\round_track_util.py:177: RuntimeWarning: invalid value encountered in double_scalars
  "distance_loaded": "{:.2%}".format(distance_loaded / distance_total),
../slevelsutil\round_track_util.py:178: RuntimeWarning: invalid value encountered in double_scalars
  "distance_rebalancing": "{:.2%}".format(distance_rebalancing / distance_total),


In [22]:
df = pd.DataFrame.from_dict(dic_all_round_track, orient='index')
df.to_csv("data1.csv")

# Getting latex table
# print(a.to_latex(multicolumn=True, multirow=True))

df

df=df.fillna(-1)
df
df.columns

Index(['instance_name', 'maximal_hiring_delay', 'batch_duration',
       'simulation_time', 'max_requests', 'initial_fleet', 'max_capacity',
       'customer_segmentation', 'contract_duration', 'service_rate',
       'allow_vehicle_hiring', 'allow_service_deterioration', 'rebalance',
       'matching', 'serviced_seats', 'picking_up_seats', 'rebalancing_seats',
       'parked_seats', 'serviced', 'denied', 'max_hired', 'occupancy', 'o1',
       'o2', 'o3', 'o4', 'v1', 'v2', 'v3', 'v4', 'mean_hired', 'median_hired',
       'avg_seats', 'max_seats', 'id_max_seats', 'median_seats', 'mean_active',
       'total_requests', 'avg_pk_delay', 'avg_ride_delay', 'total_runtime_s',
       'avg_runtime_s', 'distance_cruising', 'distance_loaded',
       'distance_rebalancing', 'distance_total'],
      dtype='object')

In [23]:

a = df#[df['instance_name']=='HIRINGWEEKMAXWAITING']

a = a[[
    'service_rate',
    'customer_segmentation',
    'contract_duration',
    'occupancy',
    'method',
    'o1',
    'o2',
    'o3',
    'o4',
    'v1',
    'v2',
    'v3',
    'v4',
    'avg_seats',
    'id_max_seats',
    'max_seats',
    'median_seats',
    "serviced_seats",
    "picking_up_seats",
    "rebalancing_seats",
    "parked_seats",
    "avg_runtime"
    
]]

KeyError: "['avg_runtime', 'method'] not in index"

In [ ]:
a.to_csv("result_fleet.csv", index=False)

In [ ]:
key_sr = 'service_rate' # (S1, S2, S3)
key_cs = 'customer_segmentation' # (A, AA, BB, etc.)
key_cd = 'contract_duration' #(0, 3600, 18000)


# Filtering data
a = a[a[key_cs].isin(["AA", "BB", "CC"])]

# convert just columns "a" and "b"
#a[['contract_duration', 'median_seats', 'distance_total']] = a[['contract_duration', 'median_seats', 'distance_total']].apply(pd.to_numeric)

a = a.astype({'contract_duration':int, 'median_seats':int, 'distance_total':int, 'max_seats': int})
# Establishing category order and alias dictionaries
dict_segmentation = {"AA":"B+", "BB":"S+", "CC":"L+", "A":"B", "B":"S", "C":"L"}
category_segmentation = pd.api.types.CategoricalDtype(categories=["B+", "S+", "L+", "B", "S", "L"], ordered=True)

dict_contract_duration = {3600:"1h", 0:"Single-ride", 10800:"3h", -1:"Baseline"}
category_contract_duration = pd.api.types.CategoricalDtype(categories=["Single-ride", "1h", "3h", "Baseline"], ordered=True)

dict_service_rate = {"S1":"SR1", "S2":"SR2", "S3":"SR3", -1:"Baseline"}
category_service_rate = pd.api.types.CategoricalDtype(categories=["SR1", "SR2", "SR3","Baseline"], ordered=True)


# Renaming data and applying aliases
a[key_cs] = a[key_cs].map(lambda e:dict_segmentation[e])
a[key_cs] = a[key_cs].astype(category_segmentation)

a[key_cd] = a[key_cd].map(lambda e:dict_contract_duration[e])
a[key_cd] = a[key_cd].astype(category_contract_duration)

a[key_sr] = a[key_sr].map(lambda e:dict_service_rate[e])
a[key_sr] = a[key_sr].astype(category_service_rate)


#a = a.sort_values(by=[key_sr, key_cs, key_cd])
#a = a.sort_values(by=[key_cd, key_cs, key_sr])
a = a.sort_values(by=[key_cs, key_cd, key_sr])



a.rename(columns={'service_rate': 'Service rate',
                    'customer_segmentation': 'Segmentation scenario',
                    'contract_duration': 'Contract duration',
                    'avg_seats': 'Avg',
                    'max_seats': 'Max',
                    'median_seats': 'Median',
                    'distance_cruising': 'Cruising',
                    'distance_rebalancing': 'Rebalancing',
                    'distance_loaded': 'Servicing',
                    'distance_total': 'Total (Km)'}, inplace=True)

#a = a.set_index(['Service rate', 'Segmentation scenario', 'Contract duration'])
#a = a.set_index(['Contract duration', 'Segmentation scenario', 'Service rate'])


a.to_csv("fleet_stats.csv")
a

In [ ]:
d = dict()
d['B+'] = 4544799
d['S+'] = 4580364
d['L+'] = 4560277

a['dif'] = a['Total (Km)']
for k,v in d.items():
    a.loc[a["Segmentation scenario"]==k, "dif"] = a.loc[a["Segmentation scenario"]==k,"dif"].apply(lambda x:x-v)

a = a.set_index(['Segmentation scenario', 'Contract duration', 'Service rate'])
a.to_csv("fleet_stats.csv")


In [ ]:
a

In [ ]:
print(a.to_latex(multicolumn=True, multirow=True))

In [ ]:
df_all = pd.DataFrame.from_dict(dic_all, orient='index')
df_all.to_csv("data1.csv")

df_all


In [ ]:
print(b.to_latex(multicolumn=True, multirow=True))

### Building indexes for multilevel table

In [ ]:

### Table: What is the service level (pickup delay) of the users lying outside SQ-class service rate?
